In [2]:
import pandas as pd
import io
from collections import Counter

def _rank_single_malignancy_type(df_input):
    # Explicitly create a copy to avoid SettingWithCopyWarning
    df_filtered = df_input.copy()

    # Create a new column for the count of datasets in the 'dataset' string
    # Handle both single datasets and combined datasets (separated by ||)
    df_filtered.loc[:, "dataset_count"] = df_filtered["dataset"].apply(
        lambda x: len(x.split("||")) if isinstance(x, str) and "||" in x else 1
    )

    ranked_dataframes_for_type = {}

    # Iterate from 1 to 7 datasets
    for count in range(1, 8):
        # Filter data for the current dataset count
        count_df = df_filtered[df_filtered["dataset_count"] == count].copy()
        
        if not count_df.empty:
            # Sort ALL results by: 1st ROC-AUC, 2nd Accuracy, 3rd Recall (all descending)
            count_df_sorted = count_df.sort_values(
                by=["test_roc_auc_ovr", "test_accuracy", "test_recall"], 
                ascending=[False, False, False]
            )
            
            # Take top 3 results overall (any model, any dataset combination with this count)
            top_3_results = count_df_sorted.head(3).copy()
            
            # Add ranking column
            top_3_results.insert(0, 'Rank', range(1, len(top_3_results) + 1))
            
            # Select and reorder columns: Rank, Dataset, ROC-AUC, Accuracy, Recall, Model
            output_df = top_3_results[['Rank', 'dataset', 'test_roc_auc_ovr', 'test_accuracy', 'test_recall', 'modelo']].copy()
            
            # Rename columns for clarity
            output_df.columns = ['Rank', 'Dataset', 'ROC-AUC', 'Accuracy', 'Recall', 'Model']
            
            # Round metrics to 2 decimal places
            output_df['ROC-AUC'] = output_df['ROC-AUC'].round(2)
            output_df['Accuracy'] = output_df['Accuracy'].round(2)
            output_df['Recall'] = output_df['Recall'].round(2)
            
            # Reset index for clean display
            output_df = output_df.reset_index(drop=True)

            ranked_dataframes_for_type[count] = output_df
            
        else:
            ranked_dataframes_for_type[count] = pd.DataFrame(columns=['Rank', 'Dataset', 'ROC-AUC', 'Accuracy', 'Recall', 'Model'])
        
    return ranked_dataframes_for_type

def get_ranked_datasets_as_dfs_by_malignancy(csv_file_path):
    df = pd.read_csv(csv_file_path)

    # Filter out rows where 'modelo' is not a string or is malformed
    df = df[df["modelo"].apply(lambda x: isinstance(x, str) and not str(x).replace(".", "").isdigit())]

    # Convert bin_malignancy to boolean (it's stored as string in CSV)
    # Also filter out any malformed rows (e.g., numeric values)
    df = df[df["bin_malignancy"].isin(['True', 'False'])].copy()
    df["bin_malignancy"] = df["bin_malignancy"] == 'True'

    all_results_by_malignancy = {}

    # Separate into two distinct dataframes based on 'bin_malignancy'
    # Explicitly create copies to avoid SettingWithCopyWarning
    df_binary = df[df["bin_malignancy"] == True].copy()
    df_multiclass = df[df["bin_malignancy"] == False].copy()

    # Process each DataFrame independently
    all_results_by_malignancy["binário"] = _rank_single_malignancy_type(df_binary)
    all_results_by_malignancy["multi-classe"] = _rank_single_malignancy_type(df_multiclass)
        
    return all_results_by_malignancy

# Example of how to use this function in a notebook:
if __name__ == "__main__":
    
    csv_path = r"modelos\resultados_modelos.csv"
    
    # First, let's check what's in the CSV
    df_check = pd.read_csv(csv_path)
    print("=== CSV DEBUG INFO ===")
    print(f"Total rows: {len(df_check)}")
    print(f"\nRaw bin_malignancy values: {df_check['bin_malignancy'].unique()}")
    
    # Clean the data
    df_check = df_check[df_check["bin_malignancy"].isin(['True', 'False'])].copy()
    df_check["bin_malignancy"] = df_check["bin_malignancy"] == 'True'
    
    print(f"\nAfter cleaning:")
    print(f"Total valid rows: {len(df_check)}")
    print(f"Binary (True): {(df_check['bin_malignancy'] == True).sum()}")
    print(f"Multi-class (False): {(df_check['bin_malignancy'] == False).sum()}")
    
    print(f"\nSample dataset names:")
    print(df_check['dataset'].head(10).tolist())
    
    print(f"\nDataset count distribution:")
    df_check['dataset_count'] = df_check["dataset"].apply(
        lambda x: len(x.split("||")) if isinstance(x, str) and "||" in x else 1
    )
    print(df_check['dataset_count'].value_counts().sort_index())
    print("=" * 50)
    print("\n")
    
    # Configure pandas to display full dataset names without truncation
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_columns', None)
    
    all_ranked_dataframes = get_ranked_datasets_as_dfs_by_malignancy(csv_path)
    
    # Also show statistics about unique dataset combinations
    print("\n=== UNIQUE DATASET COMBINATIONS PER COUNT ===")
    for count in range(1, 8):
        count_df = df_check[df_check['dataset_count'] == count]
        unique_datasets_binary = count_df[count_df['bin_malignancy'] == True]['dataset'].nunique()
        unique_datasets_multi = count_df[count_df['bin_malignancy'] == False]['dataset'].nunique()
        print(f"Count {count}: Binary={unique_datasets_binary}, Multi-class={unique_datasets_multi}")
    print("=" * 50)
    print("\n")
    
    for malignancy_type, ranked_dfs_for_type in all_ranked_dataframes.items():
        print(f"## Rankings para {malignancy_type}:\n")
        for count, df in ranked_dfs_for_type.items():
            num_results = len(df)
            print(f"### Ranking para grupos com {count} dataset(s) (showing {num_results} result(s)):\n")
            if not df.empty:
                display(df)
            else:
                print("Nenhum resultado encontrado para este grupo.\n")
            print("\n")

=== CSV DEBUG INFO ===
Total rows: 1261

Raw bin_malignancy values: ['True' 'False' '0.4444444444444444']

After cleaning:
Total valid rows: 1260
Binary (True): 630
Multi-class (False): 630

Sample dataset names:
['radiomics_exponential_scaled', 'radiomics_exponential_scaled', 'radiomics_exponential_scaled', 'radiomics_exponential_scaled', 'radiomics_exponential_scaled', 'radiomics_original_scaled', 'radiomics_original_scaled', 'radiomics_original_scaled', 'radiomics_original_scaled', 'radiomics_original_scaled']

Dataset count distribution:
dataset_count
1     70
2    200
3    350
4    350
5    210
6     70
7     10
Name: count, dtype: int64



=== UNIQUE DATASET COMBINATIONS PER COUNT ===
Count 1: Binary=7, Multi-class=7
Count 2: Binary=20, Multi-class=20
Count 3: Binary=35, Multi-class=35
Count 4: Binary=35, Multi-class=35
Count 5: Binary=21, Multi-class=21
Count 6: Binary=7, Multi-class=7
Count 7: Binary=1, Multi-class=1


## Rankings para binário:

### Ranking para grupos com 1 da

,Rank,Dataset,ROC-AUC,Accuracy,Recall,Model
0,1,radiomics_wavelet_scaled,0.93,0.88,0.78,xgboost
1,2,radiomics_wavelet_scaled,0.93,0.88,0.75,svm
2,3,radiomics_log_scaled,0.92,0.89,0.77,svm




### Ranking para grupos com 2 dataset(s) (showing 3 result(s)):



,Rank,Dataset,ROC-AUC,Accuracy,Recall,Model
0,1,radiomics_original_scaled||radiomics_wavelet_scaled,0.93,0.90,0.79,xgboost
1,2,radiomics_log_scaled||radiomics_wavelet_scaled,0.93,0.89,0.77,svm
2,3,densenet_features_nodulo_pca||radiomics_wavelet_scaled,0.93,0.87,0.80,xgboost




### Ranking para grupos com 3 dataset(s) (showing 3 result(s)):



...

,Rank,Dataset,ROC-AUC,Accuracy,Recall,Model
0,1,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_log_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.8,0.48,0.43,svm
1,2,densenet_features_nodulo_pca||radiomics_log_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.8,0.47,0.42,svm
2,3,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.8,0.47,0.40,svm




### Ranking para grupos com 6 dataset(s) (showing 3 result(s)):



,Rank,Dataset,ROC-AUC,Accuracy,Recall,Model
0,1,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_log_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.8,0.48,0.43,svm
1,2,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_logarithm_scaled||radiomics_log_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.8,0.48,0.42,svm
2,3,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_logarithm_scaled||radiomics_log_scaled||radiomics_original_scaled||radiomics_wavelet_scaled,0.8,0.47,0.41,svm




### Ranking para grupos com 7 dataset(s) (showing 3 result(s)):



,Rank,Dataset,ROC-AUC,Accuracy,Recall,Model
0,1,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_logarithm_scaled||radiomics_log_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.80,0.48,0.43,svm
1,2,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_logarithm_scaled||radiomics_log_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.79,0.46,0.45,random_forest
2,3,densenet_features_nodulo_pca||radiomics_exponential_scaled||radiomics_logarithm_scaled||radiomics_log_scaled||radiomics_original_scaled||radiomics_square_scaled||radiomics_wavelet_scaled,0.78,0.48,0.48,xgboost
